In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from typing import Union
from dataclasses import dataclass
import subprocess
import logging
from pathlib import Path
import sys
import geopandas as gpd
import rasterio as rio
from rasterio.enums import ColorInterp
from rasterio.plot import show
import numpy as np
import pandas as pd
import json
import dotenv

dotenv.load_dotenv()


scripts_dir = Path(".").joinpath("src")
if scripts_dir not in sys.path:
    sys.path.insert(0, scripts_dir.resolve().as_posix())
from helpers.mapshaper import Mapshaper
from helpers.tippcanoe import mbtileGeneration
from helpers.mapbox_uploader import uploadToMapbox
from helpers.settings import get_settings
from helpers.file_handler import FileConventionHandler
from helpers.utils import download_and_unzip_if_needed, writeReadGCP

from pipelines.processors import clean_geometries, add_names_and_translations_to_regions, split_n_parts, create_color_map, hex_to_rgb

In [2]:
mysettings = get_settings()
prev_step = "preprocess"
current_step = "tiles"

### EEZs: Exclusive Economic Zones 

In [4]:
pipe = "eez"
collection_name = f"{pipe}_v11"

eez_dir = FileConventionHandler(pipe)
# Download the EEZ file && unzip it
download_and_unzip_if_needed(eez_dir, prev_step, mysettings)

# simplify the geometries
Mapshaper(8).input([eez_dir.get_step_fmt_file_path(prev_step, "shp").as_posix()]).filter_fields(
    fields=",".join(["GEONAME", "POL_TYPE", "ISO_SOV1", "ISO_SOV2", "ISO_SOV3"])
).output(
    eez_dir.get_step_fmt_file_path(current_step, "json").as_posix(), force=True, format="geojson"
).execute()
mbtileGeneration(
    eez_dir.get_step_fmt_file_path(current_step, "json"),
    eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.mbtiles"),
)

/home/sofia/dev/skytruth-30x30/data/data/eez/processed/eez_preprocess.zip
/home/sofia/dev/skytruth-30x30/data/data/eez/processed/preprocess


Allocating 8 GB of heap memory
[o] Wrote /home/sofia/dev/skytruth-30x30/data/data/eez/processed/tiles/eez_tiles.json


PosixPath('/home/sofia/dev/skytruth-30x30/data/data/eez/processed/tiles/eez_v11.mbtiles')

In [ ]:
uploadToMapbox(
    eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

#### EEZs: Regions

In [8]:
collection_name = "regions"

# load the EEZ file & the regions file
eez_data = gpd.read_file(eez_dir.get_step_fmt_file_path(prev_step, "shp").as_posix())


with open(scripts_dir.joinpath('data_commons/data/regions_data.json'), 'r') as f:
        regions = json.load(f)

regions_df = pd.DataFrame(
    [
        {
            "region_id": data["region_iso"],
            "name": data["region_name"],
            "location_id": iso
        }
        for data in regions.get("data", [])
        for iso in data["country_iso_3s"]
    ]
)

# merge the two files
gpd.GeoDataFrame(
    pd.merge(
        eez_data,
        regions_df,
        how="left",
        left_on="ISO_SOV1",
        right_on="location_id",
        sort=True,
        copy=True,
    ),
    crs=eez_data.crs,
).to_file(
    filename=eez_dir.get_processed_step_path(prev_step)
    .joinpath(f"{pipe}_{prev_step}_{collection_name}.shp")
    .as_posix(),
    driver="ESRI Shapefile",
)

# dissolve by region_id

Mapshaper(16).input(
    [
        eez_dir.get_processed_step_path(prev_step)
        .joinpath(f"{pipe}_{prev_step}_{collection_name}.shp")
        .as_posix()
    ]
).dissolve2(fields="region_id").output(
    eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.json").as_posix(),
    force=True,
    format="geojson",
).execute()



/tmp/ipykernel_3808474/1407804161.py:34: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  ).to_file(
/home/sofia/miniforge3/envs/skytruth/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'location_id' to 'location_i'
  ogr_write(
/home/sofia/miniforge3/envs/skytruth/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Value 212881389 of field AREA_KM2 of feature 0 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
Allocating 16 GB of heap memory
[dissolve2] Dissolved 282 features into 8 features
[o] Wrote /home/sofia/dev/skytruth-30x30/data/data/eez/processed/tiles/regions.json


CompletedProcess(args='mapshaper-xl 16gb -i /home/sofia/dev/skytruth-30x30/data/data/eez/processed/preprocess/eez_preprocess_regions.shp  -dissolve2 fields=region_id -o /home/sofia/dev/skytruth-30x30/data/data/eez/processed/tiles/regions.json force format=geojson', returncode=0)

In [9]:
# Load the JSON data
with open(eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.json"), 'r') as f:
    data = json.load(f)

# Create a GeoDataFrame from the JSON data
gdf = gpd.GeoDataFrame.from_features(data['features'])

# Add names and translations to regions
gdf = add_names_and_translations_to_regions(gdf, scripts_dir)

gdf

,geometry,region_id,location,name,name_es,name_fr
1,"MULTIPOLYGON (((8.26354 -17.25, 8.25715 -17.25...",AF,3.0,Africa,África,Afrique
2,"MULTIPOLYGON (((19.38155 41.99554, 19.38144 41...",EU,6.0,Europe,Europa,Europe
3,"MULTIPOLYGON (((56.37383 24.98043, 56.38053 24...",WA,9.0,Polar,Polar,Polaire
4,"MULTIPOLYGON (((56.37383 24.98043, 56.38053 24...",WA,9.0,West Asia,Asia Occidental,Asie occidentale
5,"MULTIPOLYGON (((-56.77653 -36.29604, -56.7764 ...",SA,8.0,Latin America & Caribbean,América Latina y el Caribe,Amérique latine et Caraïbes
6,"POLYGON ((-180 -70.32232, -180 -84.36012, -179...",AT,5.0,Antartica,Antártida,Antarctique
7,"MULTIPOLYGON (((102.56807 -8.87455, 102.56638 ...",AS,4.0,Asia & Pacific,Asia y Pacífico,Asie et Pacifique
8,"MULTIPOLYGON (((-141 73.39761, -141.01268 73.3...",NA,7.0,North America,América del Norte,Amérique du Nord


In [11]:
# Save a geojson with extension json
gdf.to_file(eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.json").as_posix(), driver="GeoJSON")

/home/sofia/miniforge3/envs/skytruth/lib/python3.12/site-packages/pyogrio/raw.py:698: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(


In [68]:
# generate the mbtiles
mbtileGeneration(eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.json"))

For layer 0, using name "regions"
/home/sofia/dev/skytruth-30x30/data/data/eez/processed/tiles/regions.json:2: Found ] at top level: 
/home/sofia/dev/skytruth-30x30/data/data/eez/processed/tiles/regions.json:5: Reached EOF without all containers being closed: in JSON object {"type":"FeatureCollection","name":"regions","features":[]}
8 features, 57364566 bytes of geometry, 608 bytes of string pool
Choosing a maxzoom of -z0 for features typically 39965448 feet (12181468 meters) apart, and at least 22706313 feet (6920884 meters) apart
Choosing a maxzoom of -z9 for resolution of about 854 feet (260 meters) within features
  99.9%  9/257/169  


PosixPath('/home/sofia/dev/skytruth-30x30/data/data/eez/processed/tiles/regions.mbtiles')

In [69]:
uploadToMapbox(
    eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

upload: data/eez/processed/tiles/regions.mbtiles to s3://tilestream-tilesets-production/67/_pending/nqa72cxynns1wzq1v6pp332mc/skytruth


Linking tileset to Mapbox: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:14<00:00,  1.35s/it]


True

### Countries - gadm

In [3]:
pipe = "gadm"
collection_name = f"{pipe}_simplified"

gadm_dir = FileConventionHandler(pipe)

# Download the EEZ file && unzip it
download_and_unzip_if_needed(gadm_dir, prev_step, mysettings)

# simplify the geometries
Mapshaper(64).input([gadm_dir.get_step_fmt_file_path(prev_step, "shp").as_posix()]).filter_fields(
    fields=",".join(["GID_0", "COUNTRY", "name_es", "name_fr", 'area_km2'])
).output(
    gadm_dir.get_step_fmt_file_path(current_step, "json").as_posix(), force=True, format="geojson"
).execute()

# Generate the mbtiles
mbtileGeneration(
    gadm_dir.get_step_fmt_file_path(current_step, "json"),
    gadm_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.mbtiles"),
    True
)

/home/sofia/dev/skytruth-30x30/data/data/gadm/processed/gadm_preprocess.zip
/home/sofia/dev/skytruth-30x30/data/data/gadm/processed/preprocess


Allocating 64 GB of heap memory
[o] Wrote /home/sofia/dev/skytruth-30x30/data/data/gadm/processed/tiles/gadm_tiles.json
For layer 0, using name "gadm_tiles"
/home/sofia/dev/skytruth-30x30/data/data/gadm/processed/tiles/gadm_tiles.json:12: Found ] at top level: 
/home/sofia/dev/skytruth-30x30/data/data/gadm/processed/tiles/gadm_tiles.json:9: Reached EOF without all containers being closed: in JSON object {"type":"FeatureCollection","features":[]}
204 features, 104203392 bytes of geometry, 9732 bytes of string pool
Choosing a maxzoom of -z0 for features typically 2730281 feet (832190 meters) apart, and at least 605951 feet (184694 meters) apart
Choosing a maxzoom of -z9 for resolution of about 590 feet (180 meters) within features
  99.9%  9/146/198  


PosixPath('/home/sofia/dev/skytruth-30x30/data/data/gadm/processed/tiles/gadm_simplified.mbtiles')

In [10]:
uploadToMapbox(
    gadm_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

upload: data/gadm/processed/tiles/gadm_simplified.mbtiles to s3://tilestream-tilesets-production/96/_pending/y008s4k96pt1elm0ek7for1mc/skytruth


Linking tileset to Mapbox: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:41<00:00,  1.61s/it]


True

#### Gadm regions

In [14]:
pipe = "gadm"
gadm_dir = FileConventionHandler(pipe)
collection_name = "gadm_regions"

gadm_data = gpd.read_file(gadm_dir.get_step_fmt_file_path(prev_step, "shp").as_posix()).drop(columns=['name_es', 'name_fr'])

with open(scripts_dir.joinpath('data_commons/data/regions_data.json'), 'r') as f:
        regions = json.load(f)


regions_df = pd.DataFrame(
    [
        {
            "region_id": data["region_iso"],
            "name": data["region_name"],
            "location_id": iso
        }
        for data in regions.get("data", [])
        for iso in data["country_iso_3s"]
    ]
)

# merge the two files
gpd.GeoDataFrame(
    pd.merge(
        gadm_data,
        regions_df,
        how="left",
        left_on="GID_0",
        right_on="location_id",
        sort=True,
        copy=True,
    ),
    crs=gadm_data.crs,
).to_file(
    filename=gadm_dir.get_processed_step_path(prev_step)
    .joinpath(f"{pipe}_{prev_step}_{collection_name}.shp")
    .as_posix(),
    driver="ESRI Shapefile",
)

# dissolve by region_id 
Mapshaper(16).input(
    [
        gadm_dir.get_processed_step_path(prev_step)
        .joinpath(f"{pipe}_{prev_step}_{collection_name}.shp")
        .as_posix()
    ]
).dissolve2(fields="region_id").output(
    gadm_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.json").as_posix(),
    force=True,
    format="geojson",
).execute()


/tmp/ipykernel_3808474/4011450268.py:35: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  ).to_file(
/home/sofia/miniforge3/envs/skytruth/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'location_id' to 'location_i'
  ogr_write(
Allocating 16 GB of heap memory
[dissolve2] Dissolved 204 features into 8 features
[o] Wrote /home/sofia/dev/skytruth-30x30/data/data/gadm/processed/tiles/gadm_regions.json


CompletedProcess(args='mapshaper-xl 16gb -i /home/sofia/dev/skytruth-30x30/data/data/gadm/processed/preprocess/gadm_preprocess_gadm_regions.shp  -dissolve2 fields=region_id -o /home/sofia/dev/skytruth-30x30/data/data/gadm/processed/tiles/gadm_regions.json force format=geojson', returncode=0)

In [15]:
# Load the JSON data
with open(gadm_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.json"), 'r') as f:
    data = json.load(f)

# Create a GeoDataFrame from the JSON data
gdf = gpd.GeoDataFrame.from_features(data['features'])

# Add names and translations to regions
gdf = add_names_and_translations_to_regions(gdf, scripts_dir)

gdf

,geometry,region_id,location,name,name_es,name_fr
0,"MULTIPOLYGON (((61.283 35.609, 61.277 35.613, ...",AS,4.0,Asia & Pacific,Asia y Pacífico,Asie et Pacifique
1,"MULTIPOLYGON (((11.786 -16.78, 11.789 -16.775,...",AF,3.0,Africa,África,Afrique
2,"MULTIPOLYGON (((19.278 40.505, 19.276 40.51, 1...",EU,6.0,Europe,Europa,Europe
3,"MULTIPOLYGON (((52.444 24.107, 52.444 24.108, ...",WA,9.0,Polar,Polar,Polaire
4,"MULTIPOLYGON (((52.444 24.107, 52.444 24.108, ...",WA,9.0,West Asia,Asia Occidental,Asie occidentale
5,"MULTIPOLYGON (((-68.688 -52.61, -68.688 -52.61...",SA,8.0,Latin America & Caribbean,América Latina y el Caribe,Amérique latine et Caraïbes
6,"MULTIPOLYGON (((-169.027 -83.619, -169.029 -83...",AT,5.0,Antartica,Antártida,Antarctique
7,"MULTIPOLYGON (((-135.117 68.473, -135.119 68.4...",NA,7.0,North America,América del Norte,Amérique du Nord


In [16]:
# Save a geojson with extension json
gdf.to_file(gadm_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.json").as_posix(), driver="GeoJSON")

/home/sofia/miniforge3/envs/skytruth/lib/python3.12/site-packages/pyogrio/raw.py:698: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(


In [36]:
# generate the mbtiles
mbtileGeneration(gadm_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.json"))

For layer 0, using name "gadm_regions"
/home/sofia/dev/skytruth-30x30/data/data/gadm/processed/tiles/gadm_regions.json:2: Found ] at top level: 
/home/sofia/dev/skytruth-30x30/data/data/gadm/processed/tiles/gadm_regions.json:5: Reached EOF without all containers being closed: in JSON object {"type":"FeatureCollection","name":"gadm_regions","features":[]}
8 features, 100618534 bytes of geometry, 705 bytes of string pool
Choosing a maxzoom of -z0 for features typically 25606995 feet (7805012 meters) apart, and at least 12085379 feet (3683624 meters) apart
Choosing a maxzoom of -z9 for resolution of about 557 feet (170 meters) within features
  99.9%  9/439/254  


PosixPath('/home/sofia/dev/skytruth-30x30/data/data/gadm/processed/tiles/gadm_regions.mbtiles')

In [37]:
uploadToMapbox(
    gadm_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

upload: data/gadm/processed/tiles/gadm_regions.mbtiles to s3://tilestream-tilesets-production/4d/_pending/849nr2jzhpt1m5c3rt55232mc/skytruth


Linking tileset to Mapbox: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:25<00:00,  1.45s/it]


True

### MPAs: Marine Protected Areas from WDPA

In [ ]:
pipe = "mpa"
collection_name = "mpas_wdpa"

source_dir = FileConventionHandler(pipe)

# Download the EEZ file && unzip it
download_and_unzip_if_needed(source_dir, prev_step, mysettings)

# simplify the geometries
Mapshaper(64).input([source_dir.get_step_fmt_file_path(prev_step, "shp").as_posix()]).filter_fields(
    fields="WDPAID,NAME,PA_DEF,GIS_M_AREA,PARENT_ISO"
).clean(allow_overlaps=True, rewind=True).simplify("dp 10% keep-shapes planar").clean(
    allow_overlaps=True
).output(
    source_dir.get_step_fmt_file_path(current_step, "json").as_posix(), force=True, format="geojson"
).execute()

# generate the mbtiles
mbtileGeneration(source_dir.get_step_fmt_file_path(current_step, "json"))

In [10]:
uploadToMapbox(
    source_dir.get_step_fmt_file_path(current_step, "mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

upload: ../../data/mpa/processed/tiles/mpa_tiles.mbtiles to s3://tilestream-tilesets-production/de/_pending/yvng0dxxxru12eq9ye80350mc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [02:34<00:00,  1.54s/it]


True

### PAs: Terrestrial Protected Areas from WDPA

In [3]:
pipe = "mpa-terrestrial"
collection_name = "pas_wdpa"

source_dir = FileConventionHandler(pipe)

In [4]:
# Download the file && unzip it
download_and_unzip_if_needed(source_dir, prev_step, mysettings, "gpkg")

# split the file in two parts
wdpa = gpd.read_file(source_dir.get_step_fmt_file_path(prev_step, "gpkg").as_posix())

/home/sofia/dev/skytruth-30x30/data/data/mpa-terrestrial/processed/mpa-terrestrial_preprocess.gpkg
/home/sofia/dev/skytruth-30x30/data/data/mpa-terrestrial/processed/preprocess


In [5]:
# Divide the dataset into n parts
n_parts = 100

in_folder = source_dir.get_processed_step_path(prev_step).joinpath("parts")
in_folder.mkdir(exist_ok=True, parents=True)
folders = split_n_parts(wdpa, in_folder, n_parts)

out_folder = source_dir.get_processed_step_path(current_step).joinpath("parts")
out_folder.mkdir(exist_ok=True, parents=True)

In [6]:
# Simplify the geometries in each part
!mapshaper-xl 32gb -i /home/sofia/dev/skytruth-30x30/data/data/mpa-terrestrial/processed/preprocess/parts/*.shp  -filter-fields fields=WDPAID,NAME,PA_DEF,GIS_AREA,PARENT_ISO -clean allow-overlaps rewind -simplify dp 30% keep-shapes planar -clean allow-overlaps -o /home/sofia/dev/skytruth-30x30/data/data/mpa-terrestrial/processed/tiles/parts format=geojson

Allocating 32 GB of heap memory
[clean] Retained 2,922 of 2,922 features
[simplify] Repaired 5,750 intersections; 20 intersections could not be repaired
[clean] Retained 2,922 of 2,922 features
[o] Wrote /home/sofia/dev/skytruth-30x30/data/data/mpa-terrestrial/processed/tiles/parts/part0.json
Allocating 32 GB of heap memory
[clean] Retained 2,922 of 2,922 features
[simplify] Repaired 187 intersections
[clean] Retained 2,922 of 2,922 features
[o] Wrote /home/sofia/dev/skytruth-30x30/data/data/mpa-terrestrial/processed/tiles/parts/part10.json
Allocating 32 GB of heap memory
[clean] Retained 2,923 of 2,923 features
[simplify] Repaired 220 intersections
[clean] Retained 2,923 of 2,923 features
[o] Wrote /home/sofia/dev/skytruth-30x30/data/data/mpa-terrestrial/processed/tiles/parts/part11.json
Allocating 32 GB of heap memory
[clean] Retained 2,922 of 2,922 features
[simplify] Repaired 528 intersections; 16 intersections could not be repaired
[clean] Retained 2,922 of 2,922 features
[o] Wrot

In [7]:
# Combine simplified files into one
!mapshaper-xl 32gb -i /home/sofia/dev/skytruth-30x30/data/data/mpa-terrestrial/processed/tiles/parts/*.json combine-files -merge-layers -info -o /home/sofia/dev/skytruth-30x30/data/data/mpa-terrestrial/processed/tiles/mpa-terrestrial_tiles.json format=geojson

Allocating 32 GB of heap memory
[info] 
Layer:    part
---------------------------------------
Type:     polygon
Records:  292,258
Bounds:   -17365351.30054095,-8681816.563254528,17455418.620513726,8381220.576463436
CRS:      [unknown]
Source:   /home/sofia/dev/skytruth-30x30/data/data/mpa-terrestrial/processed/tiles/parts/part0.json

Attribute data
------------+--------------------------
 Field      | First value
------------+--------------------------
 GIS_AREA   | 218.56922190237
 NAME       | 'Laguna de los Pozuelos'
 PA_DEF     | '1'
 PARENT_ISO | 'ARG'
 WDPAID     |   3
------------+--------------------------

[o] RFC 7946 warning: non-WGS84 GeoJSON output.
[o] Wrote /home/sofia/dev/skytruth-30x30/data/data/mpa-terrestrial/processed/tiles/mpa-terrestrial_tiles.json


In [30]:
# Generate tiles
mbtileGeneration(source_dir.get_step_fmt_file_path(current_step, "json"))

For layer 0, using name "mpaterrestrial_tiles"
/home/sofia/dev/skytruth-30x30/data/data/mpa-terrestrial/processed/tiles/mpa-terrestrial_tiles.json:2292: Reached EOF without all containers being closed: in JSON object {"type":"FeatureCollection","features":[]}
/home/sofia/dev/skytruth-30x30/data/data/mpa-terrestrial/processed/tiles/mpa-terrestrial_tiles.json:8054: Found ] at top level: 
292258 features, 145131001 bytes of geometry, 14491838 bytes of string pool
Choosing a maxzoom of -z6 for features typically 16077 feet (4901 meters) apart, and at least 1742 feet (531 meters) apart
Choosing a maxzoom of -z8 for resolution of about 1487 feet (453 meters) within features
tile 0/0/0 size is 858242 with detail 12, >500000    
tile 1/0/0 size is 531410 with detail 12, >500000    
tile 1/1/0 size is 997206 with detail 12, >500000    
tile 2/1/1 size is 617557 with detail 12, >500000    
tile 2/2/1 size is 1580755 with detail 12, >500000    
tile 2/2/1 size is 814261 with detail 11, >500000   

PosixPath('/home/sofia/dev/skytruth-30x30/data/data/mpa-terrestrial/processed/tiles/mpa-terrestrial_tiles.mbtiles')

In [31]:
# Upload to mapbox
uploadToMapbox(
    source_dir.get_step_fmt_file_path(current_step, "mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

upload: data/mpa-terrestrial/processed/tiles/mpa-terrestrial_tiles.mbtiles to s3://tilestream-tilesets-production/da/_pending/ktgzxoiw6pt17hp0ixdp0s1mc/skytruth


Linking tileset to Mapbox: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:12<00:00,  1.38it/s]


True

### MPA Atlas 

In [7]:
pipe = "mpaatlas"
collection_name = "mpa_atlas"

source_dir = FileConventionHandler(pipe)
# Download the data file && unzip it if needed
download_and_unzip_if_needed(source_dir, prev_step, mysettings)

# generate the mbtiles
mbtileGeneration(
    source_dir.get_step_fmt_file_path(prev_step, "shp"),
    source_dir.get_step_fmt_file_path(current_step, "mbtiles"),
)

/home/mambauser/data/mpaatlas/processed/mpaatlas_preprocess.zip
/home/mambauser/data/mpaatlas/processed/preprocess


Allocating 50 GB of heap memory
[clean] Retained 863 of 870 features
[o] Wrote /home/mambauser/data/mpaatlas/processed/preprocess/mpaatlas_preprocess.json
For layer 0, using name "mpaatlas_preprocess"
/home/mambauser/data/mpaatlas/processed/preprocess/mpaatlas_preprocess.json:293: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
/home/mambauser/data/mpaatlas/processed/preprocess/mpaatlas_preprocess.json:21: Found ] at top level
863 features, 33449716 bytes of geometry, 29354 bytes of separate metadata, 53170 bytes of string pool
Choosing a maxzoom of -z0 for features about 282844 feet (86211 meters) apart
Choosing a maxzoom of -z12 for resolution of about 98 feet (30 meters) within features
  99.9%  12/1004/2052  


PosixPath('/home/mambauser/data/mpaatlas/processed/tiles/mpaatlas_tiles.mbtiles')

In [8]:
uploadToMapbox(
    source_dir.get_step_fmt_file_path(current_step, "mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

upload: ../../data/mpaatlas/processed/tiles/mpaatlas_tiles.mbtiles to s3://tilestream-tilesets-production/32/_pending/4py3xz71znm1h1p9cr1v050mc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [05:24<00:00,  3.25s/it]


True

### Protected seas

In [ ]:
pipe = "protectedseas"
collection_name = "protected_seas"

pipe_dir = FileConventionHandler(pipe)
input_file_moderate = pipe_dir.get_processed_step_path(prev_step).joinpath("ProtectedSeas_Navigator_LFP3_20240531.zip")
input_file_high = pipe_dir.get_processed_step_path(prev_step).joinpath("ProtectedSeas_Navigator_LFP4_5_20240531.zip")

input_file = pipe_dir.get_processed_step_path(prev_step).joinpath("protectedseas.json")
output_file = pipe_dir.get_processed_step_path(current_step).joinpath("protectedseas_tiles.mbtiles")

# Download the protected seas layers from the bucket && unzip it
writeReadGCP(
    credentials=mysettings.GCS_KEYFILE_JSON,
    bucket_name=mysettings.GCS_BUCKET,
    blob_name="ProtectedSeas/ProtectedSeas_Navigator_LFP3_20240531.zip",
    file=input_file_moderate,
    operation="r",
)

writeReadGCP(
    credentials=mysettings.GCS_KEYFILE_JSON,
    bucket_name=mysettings.GCS_BUCKET,
    blob_name="ProtectedSeas/ProtectedSeas_Navigator_LFP4_5_20240531.zip",
    file=input_file_high,
    operation="r",
)

# Load the data
protectedseas_layer_mod = gpd.read_file(input_file_moderate).pipe(clean_geometries)
protectedseas_layer_high = gpd.read_file(input_file_high).pipe(clean_geometries)


In [44]:
protectedseas_layer_high = protectedseas_layer_high[['SITE_ID', 'SITE_NAME','removal_of','geometry']]
protectedseas_layer_mod = protectedseas_layer_mod[['SITE_ID', 'SITE_NAME','removal_of','geometry']]

In [45]:
protectedseas_layer_high['FPS_cat'] = 'highly'
protectedseas_layer_mod['FPS_cat'] = 'moderately'

In [46]:
# Merge the two layers
protectedseas_layer = gpd.GeoDataFrame(
    pd.concat([protectedseas_layer_mod, protectedseas_layer_high], ignore_index=True)
)

# Save merged layer 
protectedseas_layer.to_file(input_file, driver="GeoJSON")

protectedseas_layer.head(1)

,SITE_ID,SITE_NAME,removal_of,geometry,FPS_cat
0,AIAG19,Southern No Net Zone,3,"POLYGON ((-61.82426 17.59172, -61.82393 17.591...",moderately


In [47]:
# simplify the geometries
Mapshaper(32).input([input_file.as_posix()]).clean(
    allow_overlaps=True, rewind=True
).simplify("dp 10% keep-shapes planar").clean(allow_overlaps=True).output(
    input_file.as_posix(),
    force=True,
    format="geojson",
).execute()

Allocating 32 GB of heap memory
[clean] Retained 6,741 of 6,741 features
[simplify] Repaired 5,237 intersections; 629 intersections could not be repaired
[clean] Retained 6,741 of 6,741 features
[o] Wrote /home/sofia/dev/skytruth-30x30/data/data/protectedseas/processed/preprocess/protectedseas.json


CompletedProcess(args='mapshaper-xl 32gb -i /home/sofia/dev/skytruth-30x30/data/data/protectedseas/processed/preprocess/protectedseas.json  -clean allow-overlaps rewind -simplify dp 10% keep-shapes planar -clean allow-overlaps -o /home/sofia/dev/skytruth-30x30/data/data/protectedseas/processed/preprocess/protectedseas.json force format=geojson', returncode=0)

In [48]:
# Generate mbtiles
mbtileGeneration(input_file, output_file)

For layer 0, using name "protectedseas"
/home/sofia/dev/skytruth-30x30/data/data/protectedseas/processed/preprocess/protectedseas.json:462: Found ] at top level: 
/home/sofia/dev/skytruth-30x30/data/data/protectedseas/processed/preprocess/protectedseas.json:270: Reached EOF without all containers being closed: in JSON object {"type":"FeatureCollection","features":[]}
6741 features, 9193728 bytes of geometry, 372650 bytes of string pool
Choosing a maxzoom of -z4 for features typically 73700 feet (22464 meters) apart, and at least 4638 feet (1414 meters) apart
Choosing a maxzoom of -z9 for resolution of about 755 feet (230 meters) within features
  99.9%  9/360/253  


PosixPath('/home/sofia/dev/skytruth-30x30/data/data/protectedseas/processed/tiles/protectedseas_tiles.mbtiles')

In [49]:
uploadToMapbox(
    output_file,
    "protected_seas",
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

upload: data/protectedseas/processed/tiles/protectedseas_tiles.mbtiles to s3://tilestream-tilesets-production/49/_pending/wp5zt2tnhpm1zdx09zzczs1mc/skytruth


Linking tileset to Mapbox: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:35<00:00,  1.55s/it]


True

### Terrestrial habitats

In [3]:
pipe = "terrestrial-habitats"
collection_name = "terrestrial_habitats"

pipe_dir = FileConventionHandler(pipe)
input_file = pipe_dir.get_processed_step_path(prev_step).joinpath("jung_etal_1km_reclassed.tif")
output_file = pipe_dir.get_processed_step_path(current_step).joinpath("terrestrial_habitats_tiles.mbtiles")

# Download the protected seas layers from the bucket && unzip it
writeReadGCP(
    credentials=mysettings.GCS_KEYFILE_JSON,
    bucket_name=mysettings.GCS_BUCKET,
    blob_name="terrestrial/jung_etal_2020/jung_etal_1km_reclassed.tif",
    file=input_file,
    operation="r",
)

In [4]:
# Define the land cover classes and their corresponding colors (as hex strings)
land_cover_classes = {
    1: "#01550E",          # Forest
    2: "#ffe399",          # Savanna
    3: "#C6FF53",          # Shrubland
    4: "#1D931D",          # Grassland
    5: "#5BB5FF",          # Wetlands/open water
    6: "#79685a",          # Rocky/mountains
    7: "#FBF8D6",          # Desert
    8: "#cecece",          # Artificial
    255: "#D3D3D3"         # Other
}

with rio.open(input_file) as src:
    band = src.read(1)  # Read the first band
    profile = src.profile  # Get the metadata
    crs = src.crs  # Get the CRS from the original file

# Replace 255 values with NaN
band = band.astype(np.float32)  # Change dtype to float32 to allow NaN values
band[band == 255] = np.nan  # Set 255 values to NaN

# Create the color map
color_map = create_color_map(land_cover_classes, value_to_rgb_func=hex_to_rgb)

# Create a colored image based on the band values, setting NaNs to a transparent color (for visualization only)
colored_image = np.zeros((band.shape[0], band.shape[1], 3), dtype=np.uint8)

for class_value, hex_color in land_cover_classes.items():
    if class_value != 255:  # Exclude the 'Other' category
        rgb_color = hex_to_rgb(hex_color)
        colored_image[band == class_value] = rgb_color

# Save the styled raster as a new GeoTIFF
styled_raster_path = pipe_dir.get_processed_step_path(prev_step).joinpath("jung_etal_1km_styled.tif")  # Update with desired output path

# Update profile for the RGB image
profile.update({
    'count': 3,  # Number of bands for RGB
    'dtype': 'uint8',  # Change to uint8 for RGB
    'driver': 'GTiff',  # Format
    'crs': crs,  # Add the CRS to the profile
    'compress': 'deflate',  # Apply Deflate compression
    'predictor': 2  # Use horizontal differencing predictor
})

with rio.open(styled_raster_path, 'w', **profile) as dst:
    # Write each color channel to a separate band
    dst.write(colored_image[:, :, 0], 1)  # Red channel
    dst.write(colored_image[:, :, 1], 2)  # Green channel
    dst.write(colored_image[:, :, 2], 3)  # Blue channel

    # Set color interpretation for each band
    dst.colorinterp = [ColorInterp.red, ColorInterp.green, ColorInterp.blue]

print(f"Styled raster saved at: {styled_raster_path}")

Styled raster saved at: /home/sofia/dev/skytruth-30x30/data/data/terrestrial-habitats/processed/preprocess/jung_etal_1km_styled.tif


In [74]:
remote_path = 'vizzuality_processed_data/strapi_tables/habitats.tif'
writeReadGCP(
    credentials=mysettings.GCS_KEYFILE_JSON,
    bucket_name=mysettings.GCS_BUCKET,
    blob_name=remote_path,
    file=styled_raster_path,
    operation="w",
)

In [75]:
uploadToMapbox(
    styled_raster_path,
    "terrestrial_habitats",
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

upload: data/terrestrial-habitats/processed/preprocess/jung_etal_1km_styled.tif to s3://tilestream-tilesets-production/e1/_pending/hnc0pcuglmm184g2tzay632mc/skytruth


Linking tileset to Mapbox: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:14<00:00,  1.35s/it]


True

### Terrestrial recommendations

In [86]:
pipe = "terrestrial-recommendations"
collection_name = "terrestrial_recommendations"

pipe_dir = FileConventionHandler(pipe)
input_file = pipe_dir.get_processed_step_path(current_step).joinpath("terrestrial-jung.mbtiles")

# Download the protected seas layers from the bucket && unzip it
writeReadGCP(
    credentials=mysettings.GCS_KEYFILE_JSON,
    bucket_name=mysettings.GCS_BUCKET,
    blob_name="terrestrial/jung_etal_2021/terrestial-jung.mbtiles",
    file=input_file,
    operation="r",
)

In [87]:
uploadToMapbox(
    input_file,
    "terrestrial_recommendations",
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

upload: data/terrestrial-recommendations/processed/tiles/terrestrial-jung.mbtiles to s3://tilestream-tilesets-production/3a/_pending/h47pn0cfeow1b0l2gw41502mc/skytruth


Linking tileset to Mapbox: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:20<00:00,  4.81it/s]


True

### Habitat layers

In [ ]:
# TODO: Add all the habitats layer processing
habitat = mysettings.DATA_DIR.joinpath("habitat_intermediate", "layer").resolve()

#### Warm water corals

In [ ]:
warm_water_corals = habitat.joinpath("warm_water_corals.mbtiles").resolve()

In [ ]:
uploadToMapbox(
    warm_water_corals,
    "warm_water_corals",
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

### Contextual layers

In [ ]:
# Todo: we need to add the contextual layers upload here